Importing libraries

In [24]:
import pandas as pd
from google.cloud import vision, storage, vision_v1
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\storage_key.json"

Reading csv file containing product_name and url

In [ ]:
data = pd.read_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\data.csv")
data.shape

(100, 2)

Storing url column in data_url

In [ ]:
data_url = data['product_url']
data_url

0     https://m.media-amazon.com/images/I/6176qXxgtI...
1     https://m.media-amazon.com/images/I/71XJK6bWJK...
2     https://m.media-amazon.com/images/I/81VUG3Eh5Z...
3     https://m.media-amazon.com/images/I/615wnuU9Zq...
4     https://m.media-amazon.com/images/I/71vE0KfTNQ...
                            ...                        
95    https://m.media-amazon.com/images/I/71BXLqma3B...
96    https://m.media-amazon.com/images/I/41zDXFipUm...
97    https://m.media-amazon.com/images/I/71HxhWJlza...
98    https://m.media-amazon.com/images/I/71NNd7bV8P...
99    https://m.media-amazon.com/images/I/61PIVnN9zT...
Name: product_url, Length: 100, dtype: object

Function to check if url contains a image or not

In [9]:
import requests
def is_url_image(url):
   response = requests.head(url, timeout=5)  # Use HEAD request to check headers only
   content_type = response.headers.get('Content-Type')
   if content_type and 'image' in content_type:
      return True
   else:
      return False

In [ ]:
is_url_image('https://m.media-amazon.com/images/I/71XJK6bWJKL._AC_UL480_FMwebp_QL65_.jpg')

,URL,IsImage
0,https://m.media-amazon.com/images/I/71XJK6bWJK...,True


In [ ]:
import webcolors
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

requested_colour = (119, 172, 152)
actual_name, closest_name = get_colour_name(requested_colour)

In [ ]:

def detect_properties_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.image_properties(image=image)
    props = response.image_properties_annotation
    # print("Properties:",props.dominant_colors.colors)
    # scr = [col for col in props.dominant_colors.colors]
    # print(scr)
    for color in props.dominant_colors.colors:
        try:
            r = int(color.color.red)
            g = int(color.color.green)
            b = int(color.color.blue)
            print(f"color: {get_colour_name((r,g,b))}")
        except Exception as e:
            print("gone from here",e)
            continue
        
detect_properties_uri(data_url[1])

color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'rosybrown')
color: (None, 'whitesmoke')
color: (None, 'darkslategray')
color: (None, 'black')
color: (None, 'darkslategray')
color: (None, 'gray')
color: (None, 'black')
color: (None, 'darkslategray')


Empty String variable to store results

In [3]:
txt = ""

In [10]:
sam = pd.read_csv('D:\\Work and Assignments\\Python\\sampleCSV.csv')
s_uri = sam['product_url'].tolist()
s_uri

['https://m.media-amazon.com/images/I/71xWtLhH2GL._SX695_.jpg',
 'https://m.media-amazon.com/images/I/61C+Y9zoAQL._SY695_.jpg',
 'https://m.media-amazon.com/images/I/71YG7EClYfL._SX679_.jpg',
 'https://m.media-amazon.com/images/I/81ATe15IyHL._SX695_.jpg',
 'https://m.media-amazon.com/images/I/61OBcY37KXL._SY695_.jpg',
 'https://m.media-amazon.com/images/I/710+f7XX2FL._SY695_.jpg',
 'https://m.media-amazon.com/images/I/71u2XOFXAIL._SX695_.jpg',
 'https://m.media-amazon.com/images/I/61U0BBQYhrL._SY695_.jpg',
 'https://m.media-amazon.com/images/I/71IQc8cAIQL._SX569_.jpg',
 'https://m.media-amazon.com/images/I/71fKZrHSO9L._SX569_.jpg',
 'https://m.media-amazon.com/images/I/71FAiQfnqSL._SX569_.jpg',
 'https://m.media-amazon.com/images/I/61z4pzDQROL._SX569_.jpg',
 'https://m.media-amazon.com/images/I/71N-eWQzzjL._SX569_.jpg',
 'https://m.media-amazon.com/images/I/51-FneXbwrL._SY741_.jpg',
 'https://m.media-amazon.com/images/I/51oqHcH9mcL._SY741_.jpg',
 'https://m.media-amazon.com/images/I/61

Function to extract information from image

In [11]:
def extract_info(uri):
        global txt
    
        if(is_url_image(uri)==True):
            client = vision.ImageAnnotatorClient()
            img = vision.Image()
            img.source.image_uri = uri
            
            res_label = client.label_detection(img)
            labels = res_label.label_annotations
            label = [lab.description for lab in labels]
            
            objects = client.object_localization(image=img).localized_object_annotations
            obj = [ob.name for ob in objects]
            txt = txt + f"{label}" if len(obj)==0 else txt + f"{obj}"
        
        else:
           txt = "Invalid"
        return txt

In [14]:
import concurrent.futures
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(extract_info, s_uri))

In [15]:
results

["['Shoe']['Shoe']['Person', 'Top', 'Top', 'Person']['Shoe']['Shoe']['Person', 'Top', 'Top', 'Person', 'Shirt']['Shoe']['Shoe']['Shoe']['Top', 'Person', 'Person', 'Shirt', 'Jeans']['Shoe']['Person', 'Top', 'Top', 'Person', 'Pants']['Person', 'Top', 'Person', 'Top', 'Pants']['Person', 'Top', 'Top', 'Person']['Shorts', 'Person', 'Underpants']['Pants', 'Shoe', 'Shoe']['Outerwear', 'Person']['Watch']['Pants', 'Pants', 'Pants', 'Person', 'Person', 'Person', 'Shoe', 'Shoe', 'Shoe']['Watch', 'Person', 'Person']['Watch']['Watch', 'Top', 'Pants', 'Clothing']['Watch']['Bottle', 'Bottle']['Liquid', 'Bottle', 'Fluid', 'Tints and shades', 'Plastic bottle', 'Magenta', 'Drink', 'Cylinder', 'Plastic', 'Peach']['Mobile phone', 'Mobile phone']['Top', 'Person', 'Person', 'Shirt']['Mobile phone', 'Mobile phone']['Mobile phone', 'Communication Device', 'Portable communications device', 'Gesture', 'Grey', 'Telephony', 'Gadget', 'Automotive exterior', 'Bumper', 'Smartphone']['Watch']['Mobile phone']['Mobile 

In [34]:
from google.cloud.vision_v1 import types

# Create a client
client = vision.ImageAnnotatorClient()

# Load your DataFrame
df = pd.read_csv('D:\\Work and Assignments\\Python\\sampleCSV.csv')  # Replace with your actual file

# Function to annotate images
def annotate_images(image_urls):
    requests = []
    for url in image_urls:
        image = types.Image()
        image.source.image_uri = url
        requests.append(types.AnnotateImageRequest(image=image))
    
    response = client.batch_annotate_images(requests=requests)
    return response

# Batch process the images
batch_size = 2
results = []
for i in range(0, len(df), batch_size):
    batch_urls = df['product_url'][i:i+batch_size]
    response = annotate_images(batch_urls)
    for res in response.responses:
        results.append(res)

# Extract annotations and map to DataFrame
annotations = []
for res in results:
    if res.error.message:
        annotations.append("Error: " + res.error.message)
    else:
        annotation_list = [entity.description for entity in res.label_annotations]
        annotations.append(annotation_list)

df['annotations'] = annotations


InvalidArgument: 400 Request must specify image and features.

Creating new dataframe to save results

In [ ]:
result = data
result.head()

,product_name,product_url
0,Hydro Flask Stainless Steel Wide Mouth Water B...,https://m.media-amazon.com/images/I/6176qXxgtI...
1,Owala FreeSip Insulated Stainless Steel Water ...,https://m.media-amazon.com/images/I/71XJK6bWJK...
2,"Gatorade Gx, Marble Blue, 30 Oz",https://m.media-amazon.com/images/I/81VUG3Eh5Z...
3,"Sahara Sailor Water Bottles, 32oz Motivational...",https://m.media-amazon.com/images/I/615wnuU9Zq...
4,"Insulated Water Bottle 64 oz, Triple Wall Vacu...",https://m.media-amazon.com/images/I/71vE0KfTNQ...


Iterating through function to extract features of image and adding it to result dataframe

In [ ]:
for i in range(len(data_url)):
    txt = ""
    extract_info(data_url[i])
    result.loc[i, "objects_extracted"] = txt
    

Resultant Dataframe

In [ ]:
result

,product_name,product_url,objects_extracted
0,Hydro Flask Stainless Steel Wide Mouth Water B...,https://m.media-amazon.com/images/I/6176qXxgtI...,['Vacuum flask']
1,Owala FreeSip Insulated Stainless Steel Water ...,https://m.media-amazon.com/images/I/71XJK6bWJK...,['Vacuum flask']
2,"Gatorade Gx, Marble Blue, 30 Oz",https://m.media-amazon.com/images/I/81VUG3Eh5Z...,['Bottle']
3,"Sahara Sailor Water Bottles, 32oz Motivational...",https://m.media-amazon.com/images/I/615wnuU9Zq...,"['Bottle', 'Bottle']"
4,"Insulated Water Bottle 64 oz, Triple Wall Vacu...",https://m.media-amazon.com/images/I/71vE0KfTNQ...,"['Vacuum flask', 'Bottle']"
...,...,...,...
95,Baybee R7 Battery Operated Bike for Kids,https://m.media-amazon.com/images/I/71BXLqma3B...,"['Motorcycle', 'Tire', 'Wheel', 'Tire', 'Wheel..."
96,GORMERY 1/12 Scale Model Compatible with Kawasaki,https://m.media-amazon.com/images/I/41zDXFipUm...,"['Motorcycle', 'Tire', 'Tire', 'Wheel', 'Wheel']"
97,Nexus Product Bike for Kids Toy R3 Bike,https://m.media-amazon.com/images/I/71HxhWJlza...,"['Tire', 'Wheel', 'Motorcycle', 'Wheel', 'Tire..."
98,PLUSPOINT Diecast Motorcycle Toy Ducati Bike,https://m.media-amazon.com/images/I/71NNd7bV8P...,"['Motorcycle', 'Wheel', 'Tire', 'Wheel', 'Tire..."


Saving Dataframe to CSV file

In [ ]:
result.to_csv("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv",index=False)

Uploading csv file to Cloud Stroage

In [ ]:
storage_cli = storage.Client()
bucket = storage_cli.get_bucket("bucket-shreyash")
blob = bucket.blob("Extracted_features_from_image.csv")
blob.upload_from_filename("D:\\Work and Assignments\\Python\\Assessment-2 (GOOGLE VISION API)\\results.csv")
print("completed")

completed
